# 04 - Feature Engineering

Crear features derivados (IMC, relación peso/altura, potencia de HR sobre duración, etc.) y preparar scaler para modelado.


In [ ]:
# =============================================================================
# CARGA DE DATOS PROCESADOS
# =============================================================================
# Importación de bibliotecas necesarias para feature engineering
import pandas as pd  # Manipulación de datos
import numpy as np   # Operaciones numéricas
from pathlib import Path  # Manejo de rutas
from sklearn.preprocessing import StandardScaler  # Normalización de features
import joblib  # Guardar objetos de ML (scaler)

# Definir directorio de datos procesados
DATA_DIR = Path("../data/processed")

# Cargar los tres conjuntos de datos preparados en el notebook anterior
train = pd.read_csv(DATA_DIR / 'train.csv')  # Conjunto de entrenamiento (80%)
val = pd.read_csv(DATA_DIR / 'val.csv')      # Conjunto de validación (20%)
test = pd.read_csv(DATA_DIR / 'test.csv')    # Conjunto de prueba (sin target)

In [ ]:
# =============================================================================
# CREACIÓN DE FEATURES DERIVADAS
# =============================================================================
# Generar nuevas variables a partir de las existentes para mejorar el poder predictivo
# Estas features capturan relaciones no lineales e interacciones entre variables

# Aplicar la misma ingeniería de features a los tres conjuntos de datos
for df in (train, val, test):
    # 1. Altura en metros (conversión de centímetros)
    # Necesaria para cálculo correcto del BMI
    df['Height_m'] = df['Height'] / 100.0
    
    # 2. BMI (Body Mass Index / Índice de Masa Corporal)
    # Fórmula estándar: peso (kg) / altura (m)²
    # Indicador importante de composición corporal
    df['BMI'] = df['Weight'] / (df['Height_m']**2)
    
    # 3. Frecuencia cardíaca normalizada por minuto
    # Representa la intensidad del ejercicio
    # Evitar división por cero cuando Duration = 0
    df['HR_per_min'] = df.apply(
        lambda r: r['Heart_Rate'] / (r['Duration']/60.0) if r.get('Duration', 0) > 0 else 0.0, 
        axis=1
    )
    
    # 4. Interacción HR × Duration
    # Captura el efecto combinado de intensidad (HR) y duración del ejercicio
    # Esta interacción suele ser muy predictiva del gasto calórico
    df['HRxDuration'] = df['Heart_Rate'] * df['Duration']

# Guardar versiones con feature engineering aplicado
train.to_csv(DATA_DIR / 'train_fe.csv', index=False)
val.to_csv(DATA_DIR / 'val_fe.csv', index=False)
test.to_csv(DATA_DIR / 'test_fe.csv', index=False)

print('Feature engineering complete and saved.')

In [ ]:
# =============================================================================
# NORMALIZACIÓN DE FEATURES NUMÉRICAS
# =============================================================================
# Estandarizar features para mejorar el rendimiento de modelos sensibles a escala
# (regresión lineal, KNN, redes neuronales, etc.)

# Definir lista de features numéricas a normalizar
numeric_feats = [
    'Age',           # Edad del individuo
    'Height',        # Altura en cm
    'Weight',        # Peso en kg
    'Duration',      # Duración del ejercicio (min)
    'Heart_Rate',    # Frecuencia cardíaca (BPM)
    'Body_Temp',     # Temperatura corporal (°C)
    'BMI',           # Índice de masa corporal
    'HR_per_min',    # HR normalizada por minuto
    'HRxDuration'    # Interacción HR × Duration
]

# Crear y ajustar el scaler SOLO con datos de entrenamiento
# Esto previene data leakage del conjunto de validación y prueba
scaler = StandardScaler()
scaler.fit(train[numeric_feats].fillna(0))  # Ajustar parámetros (media, std)

# Aplicar transformación a todos los conjuntos y guardar versiones escaladas
for df, fname in [(train, 'train_fe.csv'), (val, 'val_fe.csv'), (test, 'test_fe.csv')]:
    d = df.copy()  # Crear copia para no modificar el original
    # Transformar features usando los parámetros aprendidos del train
    d[numeric_feats] = scaler.transform(d[numeric_feats].fillna(0))
    # Guardar con sufijo '_scaled' para distinguir de versión sin escalar
    d.to_csv(DATA_DIR / fname.replace('.csv', '_scaled.csv'), index=False)

# Guardar el scaler para uso en producción
# Necesario para transformar nuevos datos de la misma manera
Path('../results/models').mkdir(parents=True, exist_ok=True)
joblib.dump(scaler, '../results/models/scaler.joblib')
print('Scaler saved to ../results/models/scaler.joblib')

Con features y scaler guardados, estamos listos para entrenar modelos base en `05_baseline_models.ipynb`.
